In [133]:
#测试cuda是否可用
import torch
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available.")

CUDA is available.


In [134]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
#导入数据
data_control = pd.read_excel('E:\Huawei cloud\华为云盘\文档\毕业论文\stata代码\data_control.xlsx')
data_treat = pd.read_excel('E:\Huawei cloud\华为云盘\文档\毕业论文\stata代码\data_treat.xlsx')

In [135]:
data_treat['action'] = data_treat['Year'].where(data_treat['Vc']==True)
data_treat['action'] = data_treat.groupby('Stkcd')['action'].transform('min')
treat_pre = data_treat[data_treat['Year']<data_treat['action']]
treat_post = data_treat[data_treat['Year']>=data_treat['action']]
treat_pre.head()

,Stkcd,Year,Vc,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize,action
0,12,2004,False,建筑材料,0.115335,0.481675,0.982086,0.698250,0.088428,9.292599,0.089352,0.693147,22.279432,2015.0
1,12,2005,False,建筑材料,0.105231,0.508313,0.985549,0.026604,0.077745,8.583376,0.073790,1.609438,22.432246,2015.0
2,12,2006,False,建筑材料,0.082842,0.565768,0.978328,0.092974,-0.014432,9.299277,0.067324,1.945910,22.636097,2015.0
3,12,2007,False,建筑材料,0.455311,0.493006,0.964869,0.370526,0.099847,13.703556,0.074495,2.890372,22.857785,2015.0
4,12,2008,False,建筑材料,0.123064,0.536142,0.970028,-0.118153,0.084869,15.531408,0.053134,2.197225,23.062767,2015.0


In [136]:
treat_pre = treat_pre.drop(['action'],axis=1)
df = pd.concat([data_control,treat_pre],axis=0)
df = df.drop(['Vc'],axis=1)
df.head()

,Stkcd,Year,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize
0,2,2004,房地产,0.324336,0.594163,1.0,0.612774,0.099708,12.736188,0.069947,2.833213,23.466324
1,2,2005,房地产,0.361138,0.609809,1.0,0.570613,0.113548,10.997000,0.076395,1.609438,23.813962
2,2,2006,房地产,0.981768,0.649418,1.0,0.503140,0.096677,16.854687,0.061124,3.637586,24.604993
3,2,2007,房地产,0.943422,0.661125,1.0,1.467931,0.158089,57.790683,0.070893,0.693147,25.329380
4,2,2008,房地产,0.144431,0.674441,1.0,-0.127434,0.117774,45.860878,0.042309,3.295837,25.504375


In [137]:
from sklearn.preprocessing import StandardScaler
df_r = df[['Caperc','Lev','PPEperc','NPGR','SGR','RTR','ROA','Lnsize','Lnpatent']]
scaler = StandardScaler().fit(df_r)
df[['Caperc','Lev','PPEperc','NPGR','SGR','RTR','ROA','Lnsize','Lnpatent']] = scaler.transform(df_r)
df.head()

,Stkcd,Year,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize
0,2,2004,房地产,-0.007413,0.802780,0.779348,-0.007271,0.092017,-0.029402,0.299939,0.151107,1.178013
1,2,2005,房地产,0.018697,0.878237,0.779348,-0.007382,0.121104,-0.030076,0.409323,-0.653491,1.439538
2,2,2006,房地产,0.459028,1.069261,0.779348,-0.007559,0.085647,-0.027805,0.150264,0.679959,2.034625
3,2,2007,房地产,0.431822,1.125721,0.779348,-0.005026,0.214715,-0.011935,0.315987,-1.255927,2.579576
4,2,2008,房地产,-0.135054,1.189941,0.779348,-0.009214,0.129986,-0.016560,-0.168915,0.455269,2.711224


In [138]:
#对Lnsize和Lnpatent进行缩尾处理
# 定义一个缩尾处理函数
def winsorize(series, lower_percentile=0.05, upper_percentile=0.95):
    lower_limit = np.percentile(series, lower_percentile * 100)
    upper_limit = np.percentile(series, upper_percentile * 100)
    return series.clip(lower=lower_limit, upper=upper_limit)

columns_to_winsorize = ['Lnpatent', 'Lnsize']

# 对每一列进行缩尾处理
for col in columns_to_winsorize:
    df[col] = winsorize(df[col], lower_percentile=0.05, upper_percentile=0.95)

In [139]:
from sklearn.preprocessing import LabelEncoder
label_encoder_stkcd = LabelEncoder()
df['Stkcd'] = label_encoder_stkcd.fit_transform(df['Stkcd'])
label_encoder_ind = LabelEncoder()
df['IND']= label_encoder_ind.fit_transform(df['IND'])
df.head()

,Stkcd,Year,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize
0,0,2004,11,-0.007413,0.802780,0.779348,-0.007271,0.092017,-0.029402,0.299939,0.151107,1.178013
1,0,2005,11,0.018697,0.878237,0.779348,-0.007382,0.121104,-0.030076,0.409323,-0.653491,1.439538
2,0,2006,11,0.459028,1.069261,0.779348,-0.007559,0.085647,-0.027805,0.150264,0.679959,1.866277
3,0,2007,11,0.431822,1.125721,0.779348,-0.005026,0.214715,-0.011935,0.315987,-1.255927,1.866277
4,0,2008,11,-0.135054,1.189941,0.779348,-0.009214,0.129986,-0.016560,-0.168915,0.455269,1.866277


In [140]:
num_stocks=df['Stkcd'].nunique()
print(num_stocks)

2321


In [141]:
#生成时间序列数据集
import torch
from torch.utils.data import Dataset, DataLoader
Stkcd_features = ['Stkcd']
IND_features = ['IND']
dynamic_features = ['Caperc','Lev','PPEperc','NPGR','SGR','RTR','ROA','Lnsize','Lnpatent']
class StockDataset(Dataset):
    def __init__(self, data, seq_length=3):
        self.seq_length = seq_length
        self.samples = []
        
        # 为每个股票生成所有可能的序列
        for stkcd, group in data.groupby('Stkcd'):
            group = group.sort_values('Year')
            dynamic_values = group[dynamic_features].values.astype(np.float32)
            Stkcd_values = group[Stkcd_features].iloc[0].values.astype(np.int64)
            IND_values = group[IND_features].iloc[0].values.astype(np.int64)
            
            # 生成每个时间窗口的序列
            for i in range(len(dynamic_values) - self.seq_length):
                seq = dynamic_values[i:i+self.seq_length]
                label = dynamic_values[i+self.seq_length][-1]  # 预测下一个时间点的 Lnpatent
                self.samples.append({
                    'stock_idx': Stkcd_values.item(),  # 提取标量值
                    'IND_idx': IND_values.item(),      # 提取标量值
                    'dynamic': seq,
                    'label': label
                })
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        return {
            'stock_idx': torch.tensor(sample['stock_idx'], dtype=torch.long),  # 标量值
            'IND_idx': torch.tensor(sample['IND_idx'], dtype=torch.long),      # 标量值
            'dynamic': torch.tensor(sample['dynamic'], dtype=torch.float32),
            'label': torch.tensor(sample['label'], dtype=torch.float32)
        }

In [142]:
# 创建数据集和数据加载器
dataset = StockDataset(df, seq_length=3)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 检查第一个批次
sample_batch = next(iter(dataloader))
print(f"stock_idx: {sample_batch['stock_idx']}")
print(f"IND_idx: {sample_batch['IND_idx']}")
print(f"dynamic: {sample_batch['dynamic']}")
print(f"label: {sample_batch['label']}")

stock_idx: tensor([1625, 1441])
IND_idx: tensor([13, 14])
dynamic: tensor([[[-0.2371,  0.1748,  0.0792, -0.0088, -0.0381, -0.0331, -0.2775,
          -1.1361, -0.0253],
         [-0.1894,  0.4238,  0.0819, -0.0077,  0.0048, -0.0325, -0.1319,
          -1.0038,  0.3206],
         [-0.1780,  0.8158,  0.2502, -0.0079,  0.0396, -0.0315, -0.0664,
          -0.8051,  0.6624]],

        [[ 0.0182,  0.9462,  0.6380, -0.0068,  0.1514, -0.0114,  0.1463,
           0.1878, -0.2670],
         [-0.1575,  1.2283,  0.6485, -0.0090,  0.0344, -0.0035, -0.1206,
           0.3954,  0.5023],
         [-0.3548,  1.4674,  0.6144, -0.0151, -0.4734, -0.0116, -1.8085,
           0.3872,  0.4305]]])
label: tensor([0.7612, 0.4553])


In [143]:
#划分训练集、验证集、测试集
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)
print(f"训练集样本数: {len(df_train)}")
print(f"验证集样本数: {len(df_val)}")
print(f"测试集样本数: {len(df_test)}")  

训练集样本数: 7385
验证集样本数: 1847
测试集样本数: 2309


In [144]:
# 创建数据集
train_dataset = StockDataset(df_train, seq_length=3)
val_dataset = StockDataset(df_val, seq_length=3)
test_dataset = StockDataset(df_test, seq_length=3)

In [145]:
#创建数据加载器
batch_size = 32
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=batch_size,shuffle=False)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [146]:
#定义RNN
import torch
import torch.nn as nn
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_stocks, num_industries, embedding_dim_stock, embedding_dim_industry, output_size):
        super(LSTMModel, self).__init__()
        
        # 嵌入层
        self.stock_embedding = nn.Embedding(num_stocks, embedding_dim_stock)
        self.industry_embedding = nn.Embedding(num_industries, embedding_dim_industry)
        
        # LSTM 层
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0  # 添加 Dropout
        )
        
        # 全连接层
        self.fc1 = nn.Linear(hidden_size + embedding_dim_stock + embedding_dim_industry, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, output_size)
        
        # Dropout 层
        self.dropout = nn.Dropout(0.2)
        
        # 激活函数
        self.relu = nn.ReLU()
    
    def forward(self, stock_idx, industry_idx, dynamic):
        # 嵌入层
        stock_embedded = self.stock_embedding(stock_idx)
        industry_embedded = self.industry_embedding(industry_idx)
        
        # LSTM 层
        lstm_out, _ = self.lstm(dynamic)
        lstm_last_out = lstm_out[:, -1, :]
        
        # 拼接特征
        combined = torch.cat((lstm_last_out, stock_embedded, industry_embedded), dim=1)
        
        # 全连接层
        fc1_out = self.relu(self.fc1(combined))
        fc1_out = self.dropout(fc1_out)
        fc2_out = self.relu(self.fc2(fc1_out))
        fc2_out = self.dropout(fc2_out)
        output = self.fc3(fc2_out)
        
        return output

In [147]:
# 声明模型参数
model = LSTMModel(
    input_size=9,  # 动态特征的维度
    hidden_size=64,  # LSTM 隐藏层大小
    num_layers=2,    # LSTM 层数
    num_stocks=df['Stkcd'].nunique(),  # 股票代码的数量
    num_industries=df['IND'].nunique(),  # 行业分类的数量
    embedding_dim_stock=16,  # 股票代码的嵌入维度
    embedding_dim_industry=8,  # 行业分类的嵌入维度
    output_size=1  # 输出维度
)
# 将模型移动到 GPU
device = torch.device("cuda")
model.to(device)
model

LSTMModel(
  (stock_embedding): Embedding(2321, 16)
  (industry_embedding): Embedding(31, 8)
  (lstm): LSTM(9, 64, num_layers=2, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=88, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
)

In [148]:
# 定义损失函数
criterion = nn.MSELoss()  # 均方误差损失
import torch.optim as optim
# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.01)  # 学习率可以根据需要调整
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

In [156]:
#编写train函数
def train(dataloader):
    model.train()
    
    # 初始化损失值
    running_loss = 0.0
    
    # 遍历数据加载器
    for batch_idx, batch in enumerate(dataloader):
        # 从批次中提取数据
        stock_idx = batch['stock_idx'].to(device).long()
        industry_idx = batch['IND_idx'].to(device).long()
        dynamic = batch['dynamic'].to(device)
        label = batch['label'].to(device)

        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        output = model(stock_idx, industry_idx, dynamic)
        
        # 计算损失
        loss = criterion(output, label)
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        
        # 累加损失值
        running_loss += loss.item()
    
    # 计算平均损失
    avg_loss = running_loss / len(train_dl)
    return avg_loss

In [150]:
#编写evaluate函数
def evaluate(dataloader):
    # 将模型设置为评估模式
    model.eval()
    
    # 初始化损失值
    running_loss = 0.0
    
    # 用于存储预测结果和真实值（可选）
    predictions = []
    labels = []
    
    # 禁用梯度计算
    with torch.no_grad():
        # 遍历数据加载器
        for batch_idx, batch in enumerate(val_dl):
        # 从批次中提取数据
            stock_idx = batch['stock_idx'].to(device)
            industry_idx = batch['IND_idx'].to(device)
            dynamic = batch['dynamic'].to(device)
            label = batch['label'].to(device)

            
            # 前向传播
            output = model(stock_idx, industry_idx, dynamic)
            
            # 计算损失
            loss = criterion(output, label)
            
            # 累加损失值
            running_loss += loss.item()
            
            # 记录预测结果和真实值（可选）
            predictions.extend(output.cpu().numpy())
            labels.extend(label.cpu().numpy())
    
    # 计算平均损失
    avg_loss = running_loss / len(val_dl)
    
    # 返回平均损失和预测结果（可选）
    return avg_loss, predictions, labels

In [151]:
from sklearn.metrics import mean_absolute_error, r2_score
def calculate_metrics(predictions, labels):
    predictions = np.array(predictions)
    labels = np.array(labels)
    mse = np.mean((predictions - labels) ** 2)
    mae = mean_absolute_error(labels, predictions)
    r2 = r2_score(labels, predictions)
    return mse, mae, r2

In [152]:
num_epochs = 1000
torch.manual_seed(1)

#开始训练
for epoch in range(num_epochs):
    # 训练阶段
    train_loss = train(train_dl)  # 调用 train 函数
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")
    
    # 验证阶段
    val_loss, predictions, labels = evaluate(val_dl)  # 调用 evaluate 函数
    print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}")
    
    # 更新学习率
    scheduler.step(val_loss)  # 根据验证损失调整学习率


Epoch 1/1000, Train Loss: 0.7236
Epoch 1/1000, Val Loss: 0.9034
Epoch 2/1000, Train Loss: 0.7041
Epoch 2/1000, Val Loss: 0.9136
Epoch 3/1000, Train Loss: 0.7000
Epoch 3/1000, Val Loss: 0.7390
Epoch 4/1000, Train Loss: 0.7004
Epoch 4/1000, Val Loss: 0.7680
Epoch 5/1000, Train Loss: 0.6942
Epoch 5/1000, Val Loss: 0.7950
Epoch 6/1000, Train Loss: 0.6905
Epoch 6/1000, Val Loss: 0.7651
Epoch 7/1000, Train Loss: 0.6960
Epoch 7/1000, Val Loss: 0.7479
Epoch 8/1000, Train Loss: 0.6963
Epoch 8/1000, Val Loss: 0.7782
Epoch 9/1000, Train Loss: 0.6933
Epoch 9/1000, Val Loss: 0.7811
Epoch 10/1000, Train Loss: 0.6987
Epoch 10/1000, Val Loss: 0.7769
Epoch 11/1000, Train Loss: 0.6901
Epoch 11/1000, Val Loss: 0.7749
Epoch 12/1000, Train Loss: 0.6895
Epoch 12/1000, Val Loss: 0.7722
Epoch 13/1000, Train Loss: 0.6940
Epoch 13/1000, Val Loss: 0.7719
Epoch 14/1000, Train Loss: 0.6934
Epoch 14/1000, Val Loss: 0.7663
Epoch 15/1000, Train Loss: 0.6978
Epoch 15/1000, Val Loss: 0.7669
Epoch 16/1000, Train Loss: 0

KeyboardInterrupt: 

In [ ]:
#引入transformer
class TransformerModel(nn.Module):
    def __init__(self, input_size, num_heads, num_layers, num_stocks, num_industries, embedding_dim_stock, embedding_dim_industry, output_size):
        super(TransformerModel, self).__init__()
        
        # 嵌入层
        self.stock_embedding = nn.Embedding(num_stocks, embedding_dim_stock)
        self.industry_embedding = nn.Embedding(num_industries, embedding_dim_industry)
        
        # Transformer 编码器
        self.transformer = nn.Transformer(
            d_model=input_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=512,
            dropout=0.2
        )
        
        # 全连接层
        self.fc1 = nn.Linear(input_size + embedding_dim_stock + embedding_dim_industry, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, output_size)
        
        # Dropout 层
        self.dropout = nn.Dropout(0.2)
        
        # 激活函数
        self.relu = nn.ReLU()
    
    def forward(self, stock_idx, industry_idx, dynamic):
        # 嵌入层
        stock_embedded = self.stock_embedding(stock_idx)
        industry_embedded = self.industry_embedding(industry_idx)
        
        # Transformer 编码器
        transformer_out = self.transformer(dynamic, dynamic)  # (batch_size, seq_len, input_size)
        transformer_last_out = transformer_out[:, -1, :]  # (batch_size, input_size)
        
        # 拼接特征
        combined = torch.cat((transformer_last_out, stock_embedded, industry_embedded), dim=1)
        
        # 全连接层
        fc1_out = self.relu(self.fc1(combined))
        fc1_out = self.dropout(fc1_out)
        fc2_out = self.relu(self.fc2(fc1_out))
        fc2_out = self.dropout(fc2_out)
        output = self.fc3(fc2_out)
        
        return output

In [155]:
# 声明模型参数
model = TransformerModel(
    input_size=9,  # 动态特征的维度
    num_heads = 3,    # Transformer 中注意力头的数量
    num_layers = 2,   # Transformer 的编码器和解码器层数
    num_stocks = df['Stkcd'].nunique(),   # 股票的数量
    num_industries = df['IND'].nunique(),  # 行业的数量
    embedding_dim_stock = 16,  # 股票嵌入的维度
    embedding_dim_industry = 8,  # 行业嵌入的维度
    output_size = 1  # 输出维度（回归任务）

)
# 将模型移动到 GPU
device = torch.device("cuda")
model.to(device)
model

TransformerModel(
  (stock_embedding): Embedding(2321, 16)
  (industry_embedding): Embedding(31, 8)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=9, out_features=9, bias=True)
          )
          (linear1): Linear(in_features=9, out_features=512, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=512, out_features=9, bias=True)
          (norm1): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
      )
      (norm): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-1): 2

In [157]:
# 定义损失函数
criterion = nn.MSELoss()  # 均方误差损失
import torch.optim as optim
# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.01)  # 学习率可以根据需要调整
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

In [158]:
#编写train函数
def train(dataloader):
    model.train()
    
    # 初始化损失值
    running_loss = 0.0
    
    # 遍历数据加载器
    for batch_idx, batch in enumerate(dataloader):
        # 从批次中提取数据
        stock_idx = batch['stock_idx'].to(device).long()
        industry_idx = batch['IND_idx'].to(device).long()
        dynamic = batch['dynamic'].to(device)
        label = batch['label'].to(device)

        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        output = model(stock_idx, industry_idx, dynamic)
        
        # 计算损失
        loss = criterion(output, label)
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        
        # 累加损失值
        running_loss += loss.item()
    
    # 计算平均损失
    avg_loss = running_loss / len(train_dl)
    return avg_loss

In [159]:
#编写train函数
def train(dataloader):
    model.train()
    
    # 初始化损失值
    running_loss = 0.0
    
    # 遍历数据加载器
    for batch_idx, batch in enumerate(dataloader):
        # 从批次中提取数据
        stock_idx = batch['stock_idx'].to(device).long()
        industry_idx = batch['IND_idx'].to(device).long()
        dynamic = batch['dynamic'].to(device)
        label = batch['label'].to(device)

        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        output = model(stock_idx, industry_idx, dynamic)
        
        # 计算损失
        loss = criterion(output, label)
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        
        # 累加损失值
        running_loss += loss.item()
    
    # 计算平均损失
    avg_loss = running_loss / len(train_dl)
    return avg_loss

In [160]:
from sklearn.metrics import mean_absolute_error, r2_score
def calculate_metrics(predictions, labels):
    predictions = np.array(predictions)
    labels = np.array(labels)
    mse = np.mean((predictions - labels) ** 2)
    mae = mean_absolute_error(labels, predictions)
    r2 = r2_score(labels, predictions)
    return mse, mae, r2

In [161]:
num_epochs = 1000
torch.manual_seed(1)

#开始训练
for epoch in range(num_epochs):
    # 训练阶段
    train_loss = train(train_dl)  # 调用 train 函数
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")
    
    # 验证阶段
    val_loss, predictions, labels = evaluate(val_dl)  # 调用 evaluate 函数
    print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}")
    
    # 更新学习率
    scheduler.step(val_loss)  # 根据验证损失调整学习率


Epoch 1/1000, Train Loss: 0.7690
Epoch 1/1000, Val Loss: 0.9131
Epoch 2/1000, Train Loss: 0.7057
Epoch 2/1000, Val Loss: 0.9285
Epoch 3/1000, Train Loss: 0.7009
Epoch 3/1000, Val Loss: 0.7162
Epoch 4/1000, Train Loss: 0.7008
Epoch 4/1000, Val Loss: 0.7744
Epoch 5/1000, Train Loss: 0.6941
Epoch 5/1000, Val Loss: 0.7903
Epoch 6/1000, Train Loss: 0.6904
Epoch 6/1000, Val Loss: 0.7680
Epoch 7/1000, Train Loss: 0.6963
Epoch 7/1000, Val Loss: 0.7614
Epoch 8/1000, Train Loss: 0.6960
Epoch 8/1000, Val Loss: 0.7804
Epoch 9/1000, Train Loss: 0.6932
Epoch 9/1000, Val Loss: 0.7786
Epoch 10/1000, Train Loss: 0.6987
Epoch 10/1000, Val Loss: 0.7758
Epoch 11/1000, Train Loss: 0.6902
Epoch 11/1000, Val Loss: 0.7746
Epoch 12/1000, Train Loss: 0.6896
Epoch 12/1000, Val Loss: 0.7725
Epoch 13/1000, Train Loss: 0.6942
Epoch 13/1000, Val Loss: 0.7736
Epoch 14/1000, Train Loss: 0.6934
Epoch 14/1000, Val Loss: 0.7700
Epoch 15/1000, Train Loss: 0.6979
Epoch 15/1000, Val Loss: 0.7705
Epoch 16/1000, Train Loss: 0